## 1. On-premises Algorithm
### Scalable Similarity Graph Summary

This notebook builds and queries a top-k similarity graph using two approaches:

1. **MinHashLSH-Based Graph (`ScalableSimilarityGraph`)**
   - Tokenizes text and generates MinHash signatures
   - Inserts documents into an LSH index
   - Computes true Jaccard similarity with LSH candidates
   - Maintains top-k neighbors using a heap per document

2. **Precomputed Matrix Graph (`PrecomputedSimilarityGraph`)**
   - Loads full or partial similarity matrices (e.g., from CSV)
   - Builds top-k neighbor lists from static similarity scores
   - Supports dynamic insertion of new documents using precomputed similarity vectors

Also includes utilities to:
- Preprocess text
- Load/save similarity graphs (JSON)
- Import similarity matrices (CSV)

## Assumptions

- We assume all documents and their embeddings fit on a **single machine** (i.e., non-distributed setup).
- The **similarity matrix** is precomputed using **cosine similarity** between normalized document embeddings.
- Each document has a unique ID and is compared to every other document.
- We aim to maintain a **top-k neighbor list per document**, using a min-heap to store the most similar entries.

---

### Time and Space Complexity Analysis

Let:
- `n` = number of documents
- `d` = dimensionality of each document vector (e.g., SBERT embedding)
- `k` = number of top neighbors per document (e.g., 5)

### Time Complexity

- **Cosine Similarity Matrix Computation**:
  - Dot product for all pairs of `n` vectors:  
    \[
    O(n^2 \cdot d)
    \]
- **Heap-based Top-k Construction** (per document):  
  - For each row, push `n` entries and keep top-k:  
    \[
    O(n \cdot \log k)
    \]
  - Across all documents:
    \[
    O(n^2 \cdot \log k)
    \]

➡️ **Total Time:**  
\[
\boxed{O(n^2 \cdot (d + \log k))}
\]

---

### Space Complexity

- **Embedding storage:**  
  \[
  O(n \cdot d)
  \]
- **Similarity matrix:**  
  \[
  O(n^2)
  \]
- **Top-k heaps:**  
  \[
  O(n \cdot k)
  \]

➡️ **Total Space:**  
\[
\boxed{O(n^2 + n \cdot d)}
\]

---

> ⚠️ This method is feasible only for moderate `n` (e.g., n ≤ 10,000). For larger datasets, distributed or approximate methods (e.g., Faiss, HNSW, or LSH) are recommended.


In [2]:
import heapq
import re
from datasketch import MinHash, MinHashLSH
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# --------- Utilities ---------
def preprocess(text):
    """Simple text cleaning and tokenization."""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in ENGLISH_STOP_WORDS]
    return set(tokens)

def minhash_doc(tokens, num_perm=128):
    """Generate MinHash signature for a set of tokens."""
    m = MinHash(num_perm=num_perm)
    for token in tokens:
        m.update(token.encode('utf8'))
    return m

# --------- Main Graph Class ---------
class ScalableSimilarityGraph:
    def __init__(self, threshold=0.5, top_k=5, num_perm=128):
        self.threshold = threshold
        self.top_k = top_k
        self.num_perm = num_perm
        
        self.lsh = MinHashLSH(threshold=self.threshold, num_perm=self.num_perm)
        self.doc_signatures = {}  # doc_id -> MinHash signature
        self.doc_tokens = {}  # doc_id -> token set (for true Jaccard)
        self.graph = {}  # doc_id -> min-heap of (similarity, neighbor_doc_id)
    
    def add_document(self, doc_id, text):
        """Insert a new document dynamically."""
        tokens = preprocess(text)
        signature = minhash_doc(tokens, self.num_perm)
        
        self.doc_signatures[doc_id] = signature
        self.doc_tokens[doc_id] = tokens
        self.graph[doc_id] = []  # initialize heap
        self.lsh.insert(doc_id, signature)
        
        candidates = self.lsh.query(signature)
        
        for existing_id in candidates:
            if existing_id == doc_id:
                continue
            true_sim = self.true_jaccard(doc_id, existing_id)
            
            # Update new doc's heap
            heapq.heappush(self.graph[doc_id], (true_sim, existing_id))
            if len(self.graph[doc_id]) > self.top_k:
                heapq.heappop(self.graph[doc_id])
            
            # Update existing doc's heap
            heapq.heappush(self.graph[existing_id], (true_sim, doc_id))
            if len(self.graph[existing_id]) > self.top_k:
                heapq.heappop(self.graph[existing_id])
    
    def true_jaccard(self, doc_id1, doc_id2):
        """Calculate true Jaccard similarity between two sets."""
        set1 = self.doc_tokens[doc_id1]
        set2 = self.doc_tokens[doc_id2]
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        if not union:
            return 0
        return len(intersection) / len(union)
    
    def get_top_neighbors(self, doc_id):
        """Return sorted top neighbors (highest similarity first)."""
        return sorted(self.graph.get(doc_id, []), reverse=True)

    def all_documents(self):
        """Return list of all document IDs."""
        return list(self.graph.keys())



In [3]:
class FullMatrixImporter:
    @staticmethod
    def import_full_matrix(similarity_graph, sim_matrix, doc_ids):
        """
        similarity_graph: instance of ScalableSimilarityGraph
        sim_matrix: 2D list or numpy array (full symmetric similarity matrix)
        doc_ids: list of document IDs corresponding to matrix rows/columns
        """
        n = len(doc_ids)
        
        # Initialize heaps if not already
        for doc_id in doc_ids:
            if doc_id not in similarity_graph.graph:
                similarity_graph.graph[doc_id] = []
        
        # Traverse full matrix
        for i in range(n):
            for j in range(n):
                if i == j:
                    continue  # skip self-comparison
                sim = sim_matrix[i][j]
                if sim is None:
                    continue
                doc_i = doc_ids[i]
                doc_j = doc_ids[j]
                
                # Add neighbor for doc_i
                heapq.heappush(similarity_graph.graph[doc_i], (sim, doc_j))
                if len(similarity_graph.graph[doc_i]) > similarity_graph.top_k:
                    heapq.heappop(similarity_graph.graph[doc_i])


In [4]:
import csv

def load_similarity_matrix_from_csv(file_path):
    """
    Loads a full similarity matrix from CSV.
    Returns (doc_ids, similarity_matrix)
    """
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        rows = list(reader)
        
        # First row contains column headers (skip the first blank cell)
        doc_ids = rows[0][1:]  # skip top-left blank
        
        similarity_matrix = []
        
        # Remaining rows: first column is row doc_id, rest are scores
        for row in rows[1:]:
            scores = [float(x) for x in row[1:]]  # skip row header
            similarity_matrix.append(scores)
    
    return doc_ids, similarity_matrix

In [5]:
import json

def save_graph_to_json(similarity_graph, file_path):
    output = {}
    for doc_id, neighbors in similarity_graph.graph.items():
        # Save list of neighbor tuples
        output[doc_id] = [{"neighbor": neighbor_id, "similarity": sim} for sim, neighbor_id in sorted(neighbors, reverse=True)]
    
    with open(file_path, "w") as f:
        json.dump(output, f, indent=2)

def load_graph_from_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

In [6]:
# Step 1: Load CSV
csv_file_path = "/Users/renkhunag/Desktop/Spring25/DS563/final_project/scores/similarity_matrix_full.csv"  # replace with your actual CSV file path
doc_ids, sim_matrix = load_similarity_matrix_from_csv(csv_file_path)

# Step 2: Initialize graph
similarity_graph = ScalableSimilarityGraph(threshold=0.5, top_k=5)

# Step 3: Import from loaded matrix
FullMatrixImporter.import_full_matrix(similarity_graph, sim_matrix, doc_ids)

# Step 4: Query top neighbors
for doc_id in doc_ids:
    print(f"\nTop neighbors for {doc_id}:")
    for sim, neighbor in similarity_graph.get_top_neighbors(doc_id):
        print(f" - {neighbor} (Similarity: {sim:.2f})")



Top neighbors for ita0037_0.pdf:
 - italaw7777.pdf (Similarity: 0.80)
 - ita0244.pdf (Similarity: 0.71)
 - ita0133_0.pdf (Similarity: 0.69)
 - ita0924.pdf (Similarity: 0.51)
 - ita0926.pdf (Similarity: 0.50)

Top neighbors for italaw8418.pdf:
 - italaw9532_0.pdf (Similarity: 1.00)
 - italaw9531.pdf (Similarity: 1.00)
 - italaw9530.pdf (Similarity: 1.00)
 - italaw9529_0.pdf (Similarity: 1.00)
 - italaw9528.pdf (Similarity: 1.00)

Top neighbors for italaw4264.pdf:
 - italaw4260.pdf (Similarity: 0.61)
 - italaw7399_11.pdf (Similarity: 0.53)
 - italaw7401.pdf (Similarity: 0.53)
 - italaw7306.pdf (Similarity: 0.51)
 - italaw4297.pdf (Similarity: 0.49)

Top neighbors for italaw9571.pdf:
 - italaw9590.pdf (Similarity: 0.80)
 - italaw9591_0.pdf (Similarity: 0.73)
 - italaw9574.pdf (Similarity: 0.71)
 - italaw9600_0.pdf (Similarity: 0.70)
 - italaw9575.pdf (Similarity: 0.68)

Top neighbors for italaw182159.pdf:
 - italaw182163.pdf (Similarity: 1.00)
 - italaw182160.pdf (Similarity: 1.00)
 - it

In [6]:
# Step 5: Save graph to JSON
# output_json_path = "/Users/taclin/Desktop/study/taming big data/final_project/scores/neighbor_graph.json"x
save_graph_to_json(similarity_graph, output_json_path)
print(f"\nNeighbor graph saved to {output_json_path}")


Neighbor graph saved to /Users/taclin/Desktop/study/taming big data/final_project/scores/neighbor_graph.json


In [11]:
import PyPDF2

with open('Arbitration 2.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    text = ''
    for page in reader.pages:
        text += page.extract_text()

print(text)


-----------------------------------------------------------------------------------------------------------------------------------------------------------Modak, (2008) 1 SCC 1 paras 61, 62 & 63.this judgment is protected by the law declared by the Supreme Court in Eastern Book Company v. D.B.TruePrint™ source:  Supreme Court Cases, © 2025 Eastern Book Company. The text of this version ofSCC Online Web Edition: https://www.scconline.comPrinted For: Pallavi PratapPage 1         Sunday, January 05, 2025SCC Online Web Edition, © 2025 EBC Publishing Pvt. Ltd.-----------------------------------------------------------------------------------------------------------------------------------------------------------Modak, (2008) 1 SCC 1 paras 61, 62 & 63.this judgment is protected by the law declared by the Supreme Court in Eastern Book Company v. D.B.TruePrint™ source:  Supreme Court Cases, © 2025 Eastern Book Company. The text of this version ofSCC Online Web Edition: https://www.scconline.co

In [26]:
import heapq

# --------- Main Graph Class Using Similarity Matrix ---------
class PrecomputedSimilarityGraph:
    def __init__(self, similarity_matrix, doc_ids, top_k=5):
        self.similarity_matrix = similarity_matrix
        self.doc_ids = doc_ids
        self.top_k = top_k
        self.graph = {doc_id: [] for doc_id in doc_ids}

    def initialize_graph(self):
        """Initialize heaps from the precomputed similarity matrix."""
        num_docs = len(self.doc_ids)
        
        for idx, doc_id in enumerate(self.doc_ids):
            heap = []
            for neighbor_idx, neighbor_id in enumerate(self.doc_ids):
                if neighbor_id == doc_id:
                    continue  # Skip self-similarity
                sim_score = self.similarity_matrix[idx][neighbor_idx]
                
                heapq.heappush(heap, (sim_score, neighbor_id))
                if len(heap) > self.top_k:
                    heapq.heappop(heap)  # Keep only top_k
            
            self.graph[doc_id] = heap

    def add_document(self, new_doc_id, new_similarities):
        """
        Add a new document dynamically using precomputed similarities.
        
        new_similarities: list of similarities between new_doc and existing docs.
        """
        self.graph[new_doc_id] = []

        for existing_doc_id, sim in zip(self.doc_ids, new_similarities):
            # Update new doc heap
            heapq.heappush(self.graph[new_doc_id], (sim, existing_doc_id))
            if len(self.graph[new_doc_id]) > self.top_k:
                heapq.heappop(self.graph[new_doc_id])

            # Update existing doc heap
            heapq.heappush(self.graph[existing_doc_id], (sim, new_doc_id))
            if len(self.graph[existing_doc_id]) > self.top_k:
                heapq.heappop(self.graph[existing_doc_id])

        self.doc_ids.append(new_doc_id)  # Update doc list
        self.similarity_matrix.append(new_similarities)

        # Also append similarity for new doc itself (usually 1.0)
        for i, row in enumerate(self.similarity_matrix[:-1]):
            row.append(new_similarities[i])
        self.similarity_matrix[-1].append(1.0)

    def get_top_neighbors(self, doc_id):
        """Return sorted top neighbors (highest similarity first)."""
        return sorted(self.graph.get(doc_id, []), reverse=True)

# --------- Usage Example ---------

# Assume sim_matrix is precomputed, sim_matrix[i][j] is similarity between doc_ids[i] and doc_ids[j]

# Load your precomputed similarity matrix and document IDs

# TODO: Load partial matrix from CSV
partial_matrix = sim_matrix[:3000]  # Example loading first 3000 docs
partial_doc_ids = doc_ids[:3000]

# TODO: Initialize new graph
graph = PrecomputedSimilarityGraph(partial_matrix, partial_doc_ids, top_k=5)
graph.initialize_graph()

# TODO: Insert new documents
new_doc_id = doc_ids[3000]
new_doc_similarities = sim_matrix[3000][:3000]  # Similarities of new doc with first 3000 docs
graph.add_document(new_doc_id, new_doc_similarities)

# Step 4: Query top neighbors
print(f"\nTop neighbors for {new_doc_id}:")
for sim, neighbor in graph.get_top_neighbors(new_doc_id):
    print(f" - {neighbor} (Similarity: {sim:.2f})")



Top neighbors for italaw180956.pdf:
 - italaw7432_0.pdf (Similarity: 0.66)
 - italaw180959.pdf (Similarity: 0.54)
 - italaw180957.pdf (Similarity: 0.54)
 - italaw170766.pdf (Similarity: 0.23)
 - ita1074.pdf (Similarity: 0.23)


## 2. Streaming Algorithm(first version)
## Bucketed MinHash Similarity Engine Summary

This section implements a streaming-friendly system for maintaining a top-k similarity graph using MinHash and Locality-Sensitive Hashing (LSH).

---

### Components

- **Preprocessing (`preprocess`)**  
  Cleans text, removes stopwords, and tokenizes into a set of informative words.

- **MinHash Signature (`minhash_doc`)**  
  Compresses token sets into fixed-length signatures for approximate Jaccard similarity comparison.

- **PDF Extraction (`extract_text_from_pdf`)**  
  Reads text from PDF files using PyPDF2.

---

### `ScalableSimilarityGraph` (Per-Bucket)

- Uses `MinHashLSH` to find candidate similar documents.
- Computes **true Jaccard similarity** between token sets.
- Maintains a **top-k heap** of similar neighbors per document using `heapq`.

---

### `BucketedSimilarityEngine`

- Hashes each document into one of `num_buckets` using `hash(doc_id) % num_buckets`.
- Distributes documents across multiple independent LSH graphs.
- During query:
  - Converts query text into a MinHash signature.
  - Searches all buckets for matching candidates.
  - Computes exact Jaccard similarities and returns the global top-k results.

---

### Complexity Analysis

Let:
- `n` = total number of documents
- `d` = number of unique tokens per document
- `b` = number of buckets
- `k` = number of top neighbors to maintain
- `p` = number of hash permutations in MinHash (default: 128)

**Insertion (per document):**
- Preprocessing and tokenization: `O(d)`
- MinHash signature computation: `O(p)`
- LSH insertion and query: `O(log n')` where `n' ≈ n/b` (bucket size)
- Jaccard similarity with each candidate: `O(d)`
- Heap update: `O(log k)`

**Total Insertion Time:**  
O(p + d + c·(d + log k)), where c is the number of LSH candidates


**Query Time (per query):**
- Preprocessing + MinHash: `O(p + d)`
- LSH query in all `b` buckets: `O(b · log n')`
- Candidate re-ranking by Jaccard: `O(c · d)`
- Heap-based top-k selection: `O(c · log k)`

**Space Complexity:**
- Token storage: `O(n · d)`
- MinHash signatures: `O(n · p)`
- Graph (top-k neighbors): `O(n · k)`

---

This structure allows fast, scalable, and localized similarity graph construction suitable for large document collections.


In [56]:
from datasketch import MinHashLSH, MinHash
from collections import defaultdict
import heapq
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from PyPDF2 import PdfReader
import os


# ----------------------------
# Preprocessing and MinHash
# ----------------------------
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    tokens = text.split()
    return set(token for token in tokens if token not in ENGLISH_STOP_WORDS)

def minhash_doc(tokens, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for token in tokens:
        m.update(token.encode('utf8'))
    return m

def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        print(f"[ERROR] Failed to read {pdf_path}: {e}")
        return None



# ----------------------------
# Bucketed Similarity Engine
# ----------------------------
class BucketedSimilarityEngine:
    def __init__(self, num_buckets=10, threshold=0.5, top_k=5, num_perm=128):
        self.num_buckets = num_buckets
        self.threshold = threshold
        self.top_k = top_k
        self.num_perm = num_perm
        self.buckets = [ScalableSimilarityGraph(threshold, top_k, num_perm) for _ in range(num_buckets)]

    def _bucket_index(self, doc_id):
        return hash(doc_id) % self.num_buckets

    def add_document(self, doc_id, text):
        idx = self._bucket_index(doc_id)
        self.buckets[idx].add_document(doc_id, text)

    def query(self, text, global_top_k=5):
        tokens = preprocess(text)
        signature = minhash_doc(tokens, self.num_perm)
        results = []

        # Search across all buckets
        for bucket in self.buckets:
            candidates = bucket.lsh.query(signature)
            for candidate in candidates:
                candidate_tokens = bucket.doc_tokens[candidate]
                jaccard = jaccard_tokens(tokens, candidate_tokens)
                heapq.heappush(results, (jaccard, candidate))
                if len(results) > global_top_k:
                    heapq.heappop(results)

        return sorted(results, reverse=True)

# Helper class (redefined for this cell)
class ScalableSimilarityGraph:
    def __init__(self, threshold=0.5, top_k=5, num_perm=128):
        self.threshold = threshold
        self.top_k = top_k
        self.num_perm = num_perm
        
        self.lsh = MinHashLSH(threshold=self.threshold, num_perm=self.num_perm)
        self.doc_signatures = {}  # doc_id -> MinHash signature
        self.doc_tokens = {}  # doc_id -> token set
        self.graph = defaultdict(list)  # doc_id -> min-heap of (similarity, neighbor_doc_id)

    def add_document(self, doc_id, text):
        tokens = preprocess(text)
        signature = minhash_doc(tokens, self.num_perm)

        self.doc_signatures[doc_id] = signature
        self.doc_tokens[doc_id] = tokens
        self.lsh.insert(doc_id, signature)

        candidates = self.lsh.query(signature)
        for existing_id in candidates:
            if existing_id == doc_id:
                continue
            sim = self.true_jaccard(doc_id, existing_id)
            
            # Create a new heap for doc_id
            heapq.heappush(self.graph[doc_id], (sim, existing_id))
            if len(self.graph[doc_id]) > self.top_k:
                heapq.heappop(self.graph[doc_id])

            # Update the existing doc_id
            heapq.heappush(self.graph[existing_id], (sim, doc_id))
            if len(self.graph[existing_id]) > self.top_k:
                heapq.heappop(self.graph[existing_id])

    def true_jaccard(self, doc_id1, doc_id2):
        set1 = self.doc_tokens[doc_id1]
        set2 = self.doc_tokens[doc_id2]
        if not set1 or not set2:
            return 0
        return len(set1 & set2) / len(set1 | set2)

    def get_top_neighbors(self, doc_id):
        return sorted(self.graph.get(doc_id, []), reverse=True)

def jaccard_tokens(tokens1, tokens2):
    return len(tokens1 & tokens2) / len(tokens1 | tokens2)



In [58]:
import os

# Initialize Sketch
stream_graph = BucketedSimilarityEngine(num_buckets=5, threshold=0.5, top_k=5, num_perm=128)
# read all pdf files under "docs/"

files = os.listdir('docs/')
for file in files:
    if file.endswith('.pdf'):
        file_path = 'docs/' + file
        text = extract_text_from_pdf(file_path)
        if text:
            stream_graph.add_document(doc_id=file, text=text)

# 

Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x47ee for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x48df for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x601e for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple defi

In [59]:
text = extract_text_from_pdf('docs/italaw11470.pdf')
clean_text = preprocess(text)
stream_graph.query(text)

[(1.0, 'italaw11470.pdf'),
 (0.1548140043763676, 'italaw10420.pdf'),
 (0.15257439773264053, 'italaw181037.pdf'),
 (0.14168466522678186, 'ita0287_0.pdf')]

In [66]:
for file in files:
    if file.endswith('.pdf'):
        file_path = 'docs/' + file
        text = extract_text_from_pdf(file_path)
        print(stream_graph.query(text))


[(1.0, 'italaw11470.pdf'), (0.1548140043763676, 'italaw10420.pdf'), (0.15257439773264053, 'italaw181037.pdf'), (0.14168466522678186, 'ita0287_0.pdf')]
[(1.0, 'ita0026.pdf')]
[(1.0, 'italaw16109.pdf')]
[(1.0, 'italaw181348_0.pdf')]
[(1.0, 'italaw182317.pdf'), (0.346805024576734, 'italaw7118.pdf')]
[(1.0, 'italaw170200_0.pdf')]
[(1.0, 'italaw8830.pdf')]
[(1.0, 'ita0230.pdf')]
[(1.0, 'italaw7478.pdf')]
[(1.0, 'italaw8039.pdf')]
[(1.0, 'italaw11048.pdf')]
[(1.0, 'ita0378.pdf')]
[(1.0, 'italaw11921.pdf')]
[(1.0, 'italaw9131.pdf'), (0.12969194926223143, 'italaw11299.pdf')]
[(1.0, 'italaw9657.pdf')]
[(1.0, '180439.pdf')]
[(1.0, 'italaw182061.pdf')]
[(1.0, 'italaw11538.pdf')]
[(1.0, 'italaw9848.pdf')]
[(1.0, 'italaw11299.pdf'), (0.12969194926223143, 'italaw9131.pdf')]
[(1.0, 'italaw10187.pdf')]
[(1.0, 'italaw7681.pdf')]
[(1.0, 'italaw180869.pdf')]
[(1.0, 'italaw16509.pdf')]
[(1.0, 'italaw7118.pdf'), (0.346805024576734, 'italaw182317.pdf')]
[(1.0, '180401.pdf')]
[(1.0, 'italaw16092.pdf')]
[(1.0

Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x47ee for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x48df for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x601e for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple defi

[(1.0, 'italaw1583.pdf')]


Multiple definitions in dictionary at byte 0x2100 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x3af5 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x2511 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x40ec for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x2f76 for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple d

[(1.0, 'italaw11298.pdf'), (0.1920206659012629, 'italaw7046.pdf')]
[(1.0, 'ita0619.pdf'), (0.2400073678393811, 'italaw16046.pdf')]
[(1.0, 'italaw8599.pdf')]
[(1.0, 'italaw16244.pdf')]
[(1.0, 'italaw10386.pdf')]


Overwriting cache for 0 65


[(1.0, 'italaw181989.pdf')]
[(1.0, 'italaw9452.pdf')]
[(1.0, 'italaw181037.pdf'), (0.17223738062755797, 'ita0287_0.pdf'), (0.16760450160771703, 'italaw10420.pdf'), (0.15257439773264053, 'italaw11470.pdf')]
[(1.0, 'italaw16046.pdf'), (0.2400073678393811, 'ita0619.pdf')]
[(1.0, '180166.pdf')]
[(1.0, 'italaw10219.pdf')]
[(1.0, 'italaw1146.pdf')]
[(1.0, 'italaw170421.pdf')]
[(1.0, 'ita0035.pdf')]
[(1.0, 'italaw11488.pdf')]
[(1.0, 'italaw181341.pdf'), (0.13500393597480975, 'italaw4200.pdf')]
[(1.0, 'italaw9724.pdf')]
[(1.0, 'italaw11305.pdf')]
[(1.0, 'italaw4200.pdf'), (0.13500393597480975, 'italaw181341.pdf')]
[(1.0, 'italaw16318.pdf')]
[(1.0, 'italaw3235.pdf')]
[(1.0, 'italaw9718.pdf')]
[(1.0, 'italaw7046.pdf'), (0.1920206659012629, 'italaw11298.pdf')]
[(1.0, 'italaw3220.pdf')]
[(1.0, 'ita0746.pdf')]
[(1.0, 'italaw6357_0.pdf')]
[(1.0, 'italaw10230.pdf')]
[(1.0, 'italaw170839.pdf')]
[(1.0, 'italaw170811.pdf')]
[(1.0, 'italaw182339.pdf')]
[(1.0, 'italaw10456_0.pdf')]
[(1.0, 'italaw16279.pdf

## 3. Distributed Streaming Algorithm:
Faiss + SBERT + LSH system

In [106]:
# Distributed Faiss + SBERT + LSH system: each bucket has its own Faiss index

from sentence_transformers import SentenceTransformer
from sklearn.random_projection import SparseRandomProjection
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from PyPDF2 import PdfReader
import numpy as np
import faiss
import os
import re
from collections import defaultdict

# ----------------------------
# Preprocessing
# ----------------------------
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    tokens = text.split()
    return ' '.join([token for token in tokens if token not in ENGLISH_STOP_WORDS])

def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text
    except Exception as e:
        print(f"[ERROR] Failed to read {pdf_path}: {e}")
        return None

# ----------------------------
# LSH Router
# ----------------------------
class LSHRouter:
    def __init__(self, n_planes=3):
        self.n_planes = n_planes
        self.projection = SparseRandomProjection(n_components=n_planes)

    def fit(self, vectors):
        self.projection.fit(vectors)

    def hash_vector(self, vec):
        projected = self.projection.transform([vec])[0]
        return tuple((projected > 0).astype(int))  # hash as binary vector

# ----------------------------
# Distributed Faiss Engine
# ----------------------------
class DistributedFaissEngine:
    def __init__(self, model_name='all-MiniLM-L6-v2', top_k=5, n_planes=3):
        self.model = SentenceTransformer(model_name) # load embedding model
        self.vector_size = self.model.get_sentence_embedding_dimension()
        self.top_k = top_k
        self.router = LSHRouter(n_planes=n_planes)
        self.bucket_indices = defaultdict(lambda: faiss.IndexFlatIP(self.vector_size))
        self.bucket_docs = defaultdict(list)
        self.doc_vectors = {}

    def add_documents(self, documents):
        vectors = []
        doc_ids = []

        for doc_id, text in documents:
            cleaned = preprocess_text(text)
            vec = self.model.encode(cleaned)
            vec = vec / np.linalg.norm(vec)
            vectors.append(vec)
            doc_ids.append(doc_id)
            self.doc_vectors[doc_id] = vec

        self.router.fit(np.array(vectors))

        for i, doc_id in enumerate(doc_ids):
            vec = vectors[i]
            bucket = self.router.hash_vector(vec)
            self.bucket_indices[bucket].add(np.array([vec]).astype('float32'))
            self.bucket_docs[bucket].append(doc_id)

    def query(self, text):
        cleaned = preprocess_text(text)
        query_vec = self.model.encode(cleaned)
        query_vec = query_vec / np.linalg.norm(query_vec)
        bucket = self.router.hash_vector(query_vec)

        if bucket not in self.bucket_indices:
            return []

        index = self.bucket_indices[bucket]
        doc_ids = self.bucket_docs[bucket]
        D, I = index.search(np.array([query_vec]).astype('float32'), self.top_k)

        return [(doc_ids[i], float(D[0][j])) for j, i in enumerate(I[0]) if i < len(doc_ids)]


In [107]:
docs_dir = "docs/"
stream_graph = DistributedFaissEngine(top_k=5)

for file in os.listdir(docs_dir):
    if file.endswith(".pdf"):
        file_path = os.path.join(docs_dir, file)
        text = extract_text_from_pdf(file_path)
        if text:
            # Wrap the single (doc_id, text) tuple in a list
            stream_graph.add_documents([(file, text)])


Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x47ee for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x48df for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple definitions in dictionary at byte 0x601e for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x20c for key /Subtype
Multiple defi

In [101]:
files

['.DS_Store',
 'test_data',
 'italaw11470.pdf',
 'ita0026.pdf',
 'italaw16109.pdf',
 'italaw181348_0.pdf',
 'italaw182317.pdf',
 'italaw170200_0.pdf',
 'italaw8830.pdf',
 'ita0230.pdf',
 'italaw7478.pdf',
 'italaw8039.pdf',
 'italaw11048.pdf',
 'ita0378.pdf',
 'italaw11921.pdf',
 'italaw9131.pdf',
 'italaw9657.pdf',
 '180439.pdf',
 'italaw182061.pdf',
 'italaw11538.pdf',
 'italaw9848.pdf',
 'italaw11299.pdf',
 'italaw10187.pdf',
 'italaw7681.pdf',
 'italaw180869.pdf',
 'italaw16509.pdf',
 'italaw7118.pdf',
 '180401.pdf',
 'italaw16092.pdf',
 'italaw16086.pdf',
 'italaw11931.pdf',
 'italaw10420.pdf',
 '180603.pdf',
 'italaw1294_0.pdf',
 'italaw182449.pdf',
 'italaw3008.pdf',
 'italaw7508.pdf',
 'ita0778.pdf',
 'italaw16496.pdf',
 'italaw9041.pdf',
 'italaw7737.pdf',
 'italaw8362.pdf',
 'italaw10743.pdf',
 '180574.pdf',
 'italaw11461.pdf',
 'italaw10794.pdf',
 'italaw8389.pdf',
 '180762.pdf',
 'ita0287_0.pdf',
 'italaw8883_0.pdf',
 'italaw11663.pdf',
 'italaw11105.pdf',
 'italaw8148.pdf'

In [109]:
query_file = "docs/italaw170811.pdf"
query_text = extract_text_from_pdf(query_file)

if query_text:
    results = stream_graph.query(query_text)
    for doc_id, score in results:
        print(f"{doc_id} — cosine similarity: {score:.3f}")


180574.pdf — cosine similarity: 0.706
italaw3235.pdf — cosine similarity: 0.704
italaw3008.pdf — cosine similarity: 0.625
italaw16520.pdf — cosine similarity: 0.598
italaw7737.pdf — cosine similarity: 0.595


In [103]:
results

[]